In [7]:
from PIL import Image
from PIL import ImageFont
from PIL import ImageDraw
import cv2
import numpy as np
import glob

#List tiff file
patchfile = glob.glob("*.tiff")

for impatch in patchfile:

    #Import image
    im = Image.open(impatch)

    #Set to array
    imarray = np.array(im)

    #Delete R B color
    gimarray = imarray.copy()
    gimarray[:, :, 0] = 0
    gimarray[:, :, 2] = 0
    gimarray[gimarray < 30] = 0 #Fillter noise

    #Split G color array
    gchimarray = gimarray[:, :, 1] #G chanel array ==> gray color
    cv_gchimarray = gchimarray.copy() #Bufer for cv process
    cv_gchimarray[cv_gchimarray > 0] = 100 #Set to plain gray scale

    #Calculate intentsive of green area
    gchimarray = gchimarray[gchimarray > 0] #Exiting value array
    print(impatch)
    print(gchimarray.mean())
      
    #Find&Draw contour
    _, contours, _ = cv2.findContours(cv_gchimarray, 
                                  cv2.RETR_EXTERNAL, 
                                  cv2.CHAIN_APPROX_SIMPLE)
 
    i = 1
    for c in contours:
       
        cntarea = cv2.contourArea(c)
    
        if cntarea > 500 and 1023 not in c and 0 not in c: #Improve logic!!!
        
            #Check centroid of contours
            M = cv2.moments(c)
            cX = int(M["m10"] / M["m00"])
            cY = int(M["m01"] / M["m00"])
    
            hull = cv2.convexHull(c)
        
            #Calculate Mean, IntDen
            mask = np.zeros(cv_gchimarray.shape, np.uint8)
            cv2.drawContours(mask, [hull], 0, 255, -1)
            mean_val = cv2.mean(gimarray, mask=mask)[1]
            print(i)
            print(cntarea)
            print(round(mean_val/3, 2), round(mean_val*cntarea/3, 2))
        
            #Draw Hull, Number
            cv2.drawContours(gimarray, [hull], 0, (0, 0, 255), 2)
            cv2.putText(gimarray, str(i), (cX - 10, cY), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
        
            i = i + 1

    #combine 2 image
    imarray = np.concatenate((imarray, np.full((1024, 2, 3), 255, dtype="uint8"), gimarray), axis=1)
      
    #Save image
    Finalim = Image.fromarray(imarray)
    Finalim.save("Split" + impatch[:-5] + ".jpg")


Tol4-1 high.tiff
65.41690352335496
1
718.0
11.32 8125.11
2
1202.0
22.45 26984.27
3
998.0
15.81 15779.73
4
1102.0
23.86 26292.89
5
1047.0
14.21 14881.7
6
1149.5
17.1 19655.49
7
1444.0
17.48 25242.4
8
1170.0
19.07 22311.85
9
964.5
17.91 17269.61
10
891.0
16.69 14875.19
11
1660.5
23.52 39054.57
12
1050.0
15.42 16196.23
13
1021.0
16.52 16863.91
14
1739.0
18.61 32360.14
15
1614.0
21.36 34471.64
16
1519.5
18.87 28668.38
17
971.0
19.94 19366.45
18
1440.5
23.98 34543.02
19
1158.0
19.23 22266.49
20
2057.0
20.2 41547.13
21
1017.0
22.96 23351.75
22
1467.0
19.59 28741.87
23
1282.0
20.46 26235.75
24
1539.0
17.32 26648.5
25
2011.0
23.39 47033.59
26
1947.5
26.44 51483.33
27
1075.5
23.89 25695.77
28
1270.0
18.44 23422.0
29
1086.0
23.48 25504.65
30
1605.0
20.71 33236.91
31
1957.5
22.83 44684.83
32
1551.5
21.77 33777.39
33
1253.5
22.68 28433.72
34
1016.0
9.97 10128.96
35
1166.5
21.98 25634.23
TolC251-1.tiff
52.72097145896118
1
1313.5
15.44 20279.42
2
2008.5
16.62 33388.1
3
2117.5
15.5 32817.45
4
1314.5
